In [38]:
#pip install climate_indices

In [62]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import openmeteo_requests
import requests_cache
from retry_requests import retry
from sklearn.metrics import mean_squared_error
from climate_indices import indices,

ImportError: cannot import name 'process_climate_indices' from 'climate_indices' (c:\Users\rande\anaconda3\Lib\site-packages\climate_indices\__init__.py)

In [40]:
#Importanat Definations

In [41]:
# resuable code

cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Function for frequent use for houlrly data
#https://open-meteo.com/en/docs/historical-weather-api#start_date=1950-01-01&end_date=2023-12-31&hourly=&daily=temperature_2m_mean&timezone=Europe%2FBerlin&models=era5_seamless
def daily_open_meteo_data(start_date:str, end_date:str, lat:list, long:list, variable):
	url = "https://archive-api.open-meteo.com/v1/archive"
	params = {
	"latitude": lat,
	"longitude": long,
	"start_date":start_date,
	"end_date": end_date,
	"timezone": "Europe/Berlin",
	"daily": variable,
	"models": ["era5_seamless"]
	}
	responses = openmeteo.weather_api(url, params=params)
	response = responses[0]
	# Process daily data. The order of variables needs to be the same as requested.
	daily = response.Daily()
	daily_sum = daily.Variables(0).ValuesAsNumpy()

	daily_data = {"date": pd.date_range(
	start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
	end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = daily.Interval()),
	inclusive = "left"
	)}
	daily_data["open_meteo"] = daily_sum

	daily_dataframe = pd.DataFrame(data = daily_data)
	if daily_dataframe.empty:
		return print("The daily limit reached")
	return daily_dataframe

def calculate_accuracy(original_data, predicted_data):
	mbe = np.mean(predicted_data - original_data)
	rmse = np.sqrt(mean_squared_error(original_data, predicted_data))
	correlation = np.corrcoef(original_data, predicted_data)[0, 1]

	# Print results
	print(f"Mean Bias Error (MBE): {mbe}")
	print(f"Root Mean Square Error (RMSE): {rmse}")
	print(f"Correlation: {correlation}")
	

# Function for frequent use for houlrly data
#https://open-meteo.com/en/docs/historical-weather-api#start_date=1950-01-01&end_date=2023-12-31&hourly=&daily=temperature_2m_mean&timezone=Europe%2FBerlin&models=era5_seamless
def Hourly_open_meteo_data(start_date:str, end_date:str, lat:list, long:list, variable):
	url = "https://archive-api.open-meteo.com/v1/archive"
	params = {
	"latitude": lat,
	"longitude": long,
	"start_date":start_date,
	"end_date": end_date,
	"timezone": "Europe/Berlin",
	"hourly": variable,
	"models": ["era5_seamless"]
	}
	responses = openmeteo.weather_api(url, params=params)
	response = responses[0]
	# Process daily data. The order of variables needs to be the same as requested.
	hourly = response.Hourly()
	hourly_snow_depth = hourly.Variables(0).ValuesAsNumpy()

	hourly_data = {"date": pd.date_range(
		start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
		end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
		freq = pd.Timedelta(seconds = hourly.Interval()),
		inclusive = "left"
	)}
	hourly_data["open_meteo"] = hourly_snow_depth

	hourly_dataframe = pd.DataFrame(data = hourly_data)

	if hourly_dataframe.empty:
		return print("The daily limit reached")
	return hourly_dataframe



# plots
def plot_my_bargraph(length:int,breath:int,xaxis,yaxis,title:str,xlabel:str,ylabel:str, widthv=0.5, colorv='blue',xtick = False):
    plt.figure(figsize=(length, breath))
    plt.bar(xaxis, yaxis, width = widthv , color=colorv)
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.grid(axis='y')
    if xtick == True:
        plt.xticks(xaxis,rotation=45,fontsize=10)
    plt.tight_layout()
    plt.show()



def plot_my_bargraph_withnumbers(length:int,breath:int,xaxis,yaxis,title:str,xlabel:str,ylabel:str, widthv=0.5, colorv='blue'):
    plt.figure(figsize=(length, breath))
    plt.bar(xaxis, yaxis, width = widthv , color=colorv)
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.grid(axis='y')
    for i, value in enumerate(yaxis):
        plt.text(xaxis[i], value + 1, str(value), ha='center', fontsize=9)
    plt.tight_layout()
    plt.xticks(xaxis,rotation=45,fontsize=10)
    plt.show()

## calcluating SEPI index

### Vicente-Serrano, S. M., Beguería, S., & López-Moreno, J. I. (2010). "A Multiscalar Drought Index Sensitive to Global Warming: The Standardized Precipitation Evapotranspiration Index." Journal of Climate, 23(7), 1696–1718. https://doi.org/10.1175/2009JCLI2909.1

### Projecting Hydroclimatic Extremes: Climate Change Impacts on Drought in a German Low Mountain Range Catchment



# Steps 
### 1. Get the data from the API.
### 2. Convert it to monthly (Monthly Mean)
### 3. Calcluate D (D = p - et0)
### 4. Normalize it using Climate Indicies library python.
### plot the result. 


In [42]:
open_meto_1950_2023_precipitation = daily_open_meteo_data("1950-01-01", "2023-12-31",[51.8], [10.62],"precipitation_sum")

In [43]:
open_meto_1950_2023_et0 = daily_open_meteo_data("1950-01-01", "2023-12-31",[51.8], [10.62],"et0_fao_evapotranspiration")

### Converting it into monthly data

In [44]:
# Precipitation data
open_meto_1950_2023_precipitation.set_index('date',inplace = True)
open_meto_1950_2023_precipitation.fillna(0,inplace=True) # to be sure it do not contain NAN values
open_meto_1950_2023_precipitation_monthly = open_meto_1950_2023_precipitation.resample("M").mean()
open_meto_1950_2023_precipitation_monthly.rename(columns={'open_meteo': 'precipitation'}, inplace=True)

In [45]:
open_meto_1950_2023_precipitation_monthly.head()

,precipitation
date,
1949-12-31 00:00:00+00:00,0.000000
1950-01-31 00:00:00+00:00,1.922581
1950-02-28 00:00:00+00:00,3.639286
1950-03-31 00:00:00+00:00,1.070968
1950-04-30 00:00:00+00:00,2.833333


In [46]:
# Eto data
open_meto_1950_2023_et0.set_index('date',inplace = True)
open_meto_1950_2023_et0.fillna(0,inplace=True) # to be sure it do not contain NAN values
open_meto_1950_2023_eto_monthly = open_meto_1950_2023_et0.resample("M").mean()
open_meto_1950_2023_eto_monthly.rename(columns={'open_meteo': 'ET0'}, inplace=True)

In [47]:
open_meto_1950_2023_eto_monthly.head()

,ET0
date,
1949-12-31 00:00:00+00:00,0.376717
1950-01-31 00:00:00+00:00,0.352772
1950-02-28 00:00:00+00:00,0.540628
1950-03-31 00:00:00+00:00,1.021201
1950-04-30 00:00:00+00:00,1.453352


### Calculating water balance (WB)

In [48]:
WB_1950_2023_eto_monthly = open_meto_1950_2023_eto_monthly

In [49]:
WB_1950_2023_eto_monthly['Deficit'] = open_meto_1950_2023_precipitation_monthly['precipitation'] - open_meto_1950_2023_eto_monthly['ET0']

### Using the climate-indices library to calcluate the SPEI.
### Climate-indices: https://pypi.org/project/climate_indices/


### Citation
### author = "James Adams",
### title  = "climate_indices, an open source Python library providing reference implementations of commonly used climate indices",
### url    = "https://github.com/monocongo/climate_indices",
### month  = "may",
### year   = "2017--"

In [65]:
## defination for Calculating the SPEI
def compute_SPEI(distribution,scale,pet,precips,name_field):
    spei_values = indices.spei(
    pet_mm= pet,
    precips_mm= precips,
    scale=scale,
    distribution=distribution,
    data_start_year=1950,
    calibration_year_initial=1950,
    calibration_year_final=2023,
    periodicity='monthly',
)
    return spei_values


#data[name_field] = spei_values

In [66]:
 # Use Gamma distribution, and 3 month scale
distribution = indices.Distribution.gamma 
scale = 3 

compute_SPEI(distribution,scale,WB_1950_2023_eto_monthly,'SPEI_3_month')

TypeError: spei() got an unexpected keyword argument 'data'

In [ ]:
WB_1950_2023_eto_monthly['SPEI_3_month'].plot(figsize=(12, 6), title="3-Month SPEI")
plt.axhline(0, color='black', linestyle='--')
plt.xlabel('Year') 
plt.ylabel('SPEI')
plt.show()